# Data Visualization
This notebook was used for generating data visualizations for the social media analysis of the Netherlans eScience Center in the team sprint of September 2018. The first bar graphs show the total of tweets per year for a certain topic. Year bars are divided in colored parts which specify what percentage of the tweets of that year was written in which language.  Graphs were generated for three different files (not supplied) here):
1. others-search.csv: accounts other than @esciencecenter that tweeted about esciencecenter
2. esciencecenter-user.csv: tweets from the user @esciencecenter
3. nlesc-search.csv: tweets that contained the string nlesc

In [ ]:
import csv
from datetime import datetime
import sys

DATADIR = "../../data"
INFILE = "others-search.csv" # or: others-search.csv esciencecenter-user.csv nlesc-search.csv 
LANGS = ["en","nl","other","unknown"]
inFile = open(DATADIR+"/"+INFILE,"r")
csvreader = csv.DictReader(inFile)

table = {}
for row in csvreader:
    lang = row["lang"]
    if lang == "" or lang == "und": lang = "unknown"
    elif not lang == "en" and not lang == "nl": lang = "other"
    year = datetime.strptime(row["date"],"%Y-%m-%d").year
    if not lang in table: 
        for l in LANGS: table[l] = {}
    if not year in table[lang]: 
        for l in LANGS: table[l][year] = 0
    table[lang][year] += 1
inFile.close()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import operator

font = {'size':16}
matplotlib.rc('font', **font)

plt.figure(figsize=(15,8))
en = plt.bar(list(table["en"].keys()),list(table["en"].values()),label="English")
nl = plt.bar(list(table["nl"].keys()),list(table["nl"].values()),color="r",
             bottom=list(table["en"].values()),label="Dutch")
other = plt.bar(list(table["other"].keys()),list(table["other"].values()),color="g",
                bottom=list(map(operator.add,list(table["en"].values()),list(table["nl"].values()))),
                label="Other")
unknown = plt.bar(list(table["unknown"].keys()),list(table["unknown"].values()),color="orange",
                  bottom=list(map(operator.add,list(map(operator.add,list(table["en"].values()),
                                                list(table["nl"].values()))),
                                  list(table["other"].values()))),label="Unknown")
plt.title(INFILE)
plt.legend(handles=[en,nl,other,unknown])
plt.savefig(INFILE+".png")
plt.show()

Since the total number of tweets is not constant, we should compare the tweet counts for our targets to overall counts. These are not available but we have an approximation of the total number of Dutch tweets per month. These were used for generating the next graph: "Dutch tweets in millions per month". The counts were obtained from the website twiqs.nl from the file http://145.100.59.92/counts/months.txt (the ip address may change in the future).

In [ ]:
from matplotlib import dates

INFILE = "twiqs-counts.txt"
try: inFile = open(DATADIR+"/"+INFILE,"r")
except: sys.exit("opening file "+INFILE+"failed")
    
table = {}
for line in inFile:
    fields = line.split()
    thisDate = datetime.strptime(fields[0],"%Y%m")
    month = thisDate.replace(day=1)
    monthNum = dates.date2num(month)
    if not monthNum in table: table[monthNum] = 0
    table[monthNum] += int(fields[1])
inFile.close()

In [ ]:
thisPlot = plt.figure(figsize=(15,8))
axis = thisPlot.add_subplot(111)
axis.xaxis.set_major_formatter(dates.DateFormatter('%m/%Y'))
plt.plot(list(table.keys()),list(table.values()))
plt.title("Dutch tweets (in millions per month)")
plt.savefig(INFILE+".png")
plt.show()

Finally, we counted how many tweets send by @esciencecenter used one of the hastags of our yearly events: #nlesc2013 #nlesc14 escience2015 nlesc15 escience16 nlesc16 nlesc17 and escience2018. The total counts for each year were displayed in a bar graph.  

In [ ]:
eventCountsPerYear = {2013:53,2014:14,2015:54,2016:104,2017:26,2018:2}
plt.figure(figsize=(15,8))
plt.bar(list(eventCountsPerYear.keys()),list(eventCountsPerYear.values()))
plt.savefig("eventCountsPerYear.png")
plt.show()